In [1]:
from batchcreator import DataGenerator, get_list_IDs
from datetime import datetime
import tensorflow as tf
from model_builder import build_generator, GAN
import numpy as np

import matplotlib.pyplot as plt
import logger

In [2]:
import wandb
from wandb.keras import WandbCallback

In [3]:
run = wandb.init(project='precipitation-forecasting',
                config={
                    'batch_size': 1,
                    'epochs': 5,
                    'lr_g': 0.001,
                    'lr_d': 0.0001,
                    'x_length': 6,
                    'y_length': 1,
                    'rnn_type': 'GRU',
                    'filter_no_rain': True,
                    'g_model': 'AENN'
                })
config_wandb = wandb.config

wandb: Currently logged in as: koert (use `wandb login --relogin` to force relogin)


In [4]:
start_dt = datetime(2019,6,6,0,0)
end_dt =  datetime(2019,6,8,1,0)

list_IDs = get_list_IDs(start_dt, end_dt, config_wandb.x_length, config_wandb.y_length, filter_no_rain=config_wandb.filter_no_rain)
print(list_IDs[-1])
print(len(list_IDs))

(['201906080000', '201906080005', '201906080010', '201906080015', '201906080020', '201906080025'], ['201906080030'])
76


In [5]:
same_data = [list_IDs[0] for i in range(0, 32)]
# same_data = [list_IDs[0],  list_IDs[0], list_IDs[0], list_IDs[0], list_IDs[0], list_IDs[0], list_IDs[0], list_IDs[0]]
print(same_data[0])
print(len(same_data))

(['201906052330', '201906052335', '201906052340', '201906052345', '201906052350', '201906052355'], ['201906060000'])
32


In [6]:
model = GAN(rnn_type=config_wandb.rnn_type, x_length=config_wandb.x_length, 
            y_length=config_wandb.y_length, architecture=config_wandb.g_model, relu_alpha=.2)
model.compile(lr_g=config_wandb.lr_g, lr_d=config_wandb.lr_d)

In [7]:
# model = build_generator(rnn_type=config_wandb.rnn_type, x_length=config_wandb.x_length, 
#             y_length=config_wandb.y_length, architecture=config_wandb.g_model, relu_alpha=.2)

# from tensorflow.keras.optimizers import Adam

# opt = Adam(learning_rate=0.01)
# model.compile(loss = tf.keras.losses.MeanSquaredError(),  
#    optimizer = opt, metrics=['mae'])

In [8]:
# val data:
start_dt = datetime(2019,6,5,0,0)
end_dt =  datetime(2019,6,8,1,0)

list_IDs = get_list_IDs(start_dt, end_dt, config_wandb.x_length, config_wandb.y_length, 
                        filter_no_rain=config_wandb.filter_no_rain)

generator_val = DataGenerator(list_IDs, batch_size=config_wandb.batch_size, x_seq_size=config_wandb.x_length, 
                                       y_seq_size=config_wandb.y_length, norm_method=None, load_from_npy=True)

x_test = []
y_test = []

for xs, ys in generator_val:
    x_test.extend(xs)
    y_test.extend(ys)
x_test = np.array(x_test)
y_test = np.array(y_test)
# x_test = tf.convert_to_tensor(x_test)
# y_test = tf.convert_to_tensor(y_test)
print(x_test.shape)
print(y_test.shape)

(123, 6, 768, 700, 1)
(123, 1, 384, 350, 1)


In [9]:
# from tensorflow.keras import backend as K

# # Define the Gradient Fucntion
# epoch_gradient = []

# def get_gradient_func(model):
#     grads = K.gradients(model.total_loss, model.trainable_weights)
#     inputs = model._feed_inputs + model._feed_targets + model._feed_sample_weights
#     func = K.function(inputs, grads)
#     return func

# # Define the Required Callback Function
# class GradientCalcCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs=None):
#       get_gradient = get_gradient_func(model)
#       grads = get_gradient([test_images, test_labels, np.ones(len(test_labels))])
#       epoch_gradient.append(grads)

In [10]:
    def test_logger(x_test, y_test, n=4):
        indexes = np.random.randint(0, len(x_test), n)

        xs = x_test[indexes]
        ys = y_test[indexes]
        
        batch_size = tf.shape(xs)[0]

        # Decode them to fake images
        generated_images = model.generator(xs)

        # Combine them with real images
        combined_images = tf.concat([generated_images, ys], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        
        # Add random noise to the labels - important trick!
        #labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = model.discriminator(combined_images)
            d_loss = model.loss_fn(labels, predictions)
        grads_d = tape.gradient(d_loss, model.discriminator.trainable_weights)

       
      
        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = model.discriminator(model.generator(xs))
            g_loss_gan = model.loss_fn(misleading_labels, predictions)
     
           # g_loss_mse = self.loss_mse(ys, predictions)
            g_loss = g_loss_gan #+ g_loss_mse
        
        grads_g = tape.gradient(g_loss, model.generator.trainable_weights)
        print(len(grads_g))
        return grads_g
#         print(len(grads_g))
#         #print(tf.shape(grads_g))
#         grads_g = tf.reshape(grads_g)
#         print(len(grads_g))
#         print(grads_g)
 
#         wandb.log({'grads_g': wandb.Histogram(grads_g)})


In [11]:
grads_g = test_logger(x_test,y_test)

20


In [12]:
x_test.shape

(123, 6, 768, 700, 1)

In [ ]:
generator = DataGenerator(list_IDs, batch_size=32, x_seq_size=config_wandb.x_length, 
                                       y_seq_size=config_wandb.y_length, norm_method=None, load_from_npy=True)

wandb_callback = wandb.keras.WandbCallback()

hist = model.fit(generator, epochs=30, 
                 callbacks=[wandb_callback, 
                            logger.GradientLogger(x_test,y_test), 
                            logger.ImageLogger(x_test,y_test)
                           ])

Epoch 1/30
3/3 [==============================] - 30s 9s/step - d_loss: 0.6707 - g_loss: 0.6936 - mse_loss: 0.0000e+00
Epoch 2/30
3/3 [==============================] - 28s 9s/step - d_loss: 0.6426 - g_loss: 0.6793 - mse_loss: 0.0000e+00
Epoch 3/30
3/3 [==============================] - 28s 9s/step - d_loss: 0.6660 - g_loss: 0.6436 - mse_loss: 0.0000e+00
Epoch 4/30
3/3 [==============================] - 28s 9s/step - d_loss: 0.6734 - g_loss: 0.6106 - mse_loss: 0.0000e+00
Epoch 5/30
3/3 [==============================] - 28s 9s/step - d_loss: 0.6852 - g_loss: 0.6032 - mse_loss: 0.0000e+00
Epoch 6/30
3/3 [==============================] - 28s 9s/step - d_loss: 0.6746 - g_loss: 0.6712 - mse_loss: 0.0000e+00
Epoch 7/30
3/3 [==============================] - 28s 9s/step - d_loss: 0.6098 - g_loss: 0.8189 - mse_loss: 0.0000e+00
Epoch 8/30


In [ ]:
generator = DataGenerator(same_data, batch_size=8, x_seq_size=config_wandb.x_length, 
                                       y_seq_size=config_wandb.y_length, norm_method=None, load_from_npy=True)

wandb_callback = wandb.keras.WandbCallback()
#training_data=(val_x, val_y), log_gradients=True
hist = model.fit(generator, epochs=15, callbacks=[wandb_callback, logger.ImageLogger(x_test,y_test)])

In [ ]:
xs_in = []
ys_out = []
for xs,ys in generator:
   # print(xs.shape, ys.shape)
    xs_in.append(xs)
    ys_out.append(ys)
ys_out = np.array(ys_out)
xs_in = np.array(xs_in)

print(xs_in.shape)
print(ys_out.shape)

In [ ]:
plt.imshow(np.squeeze(ys_out[0][0]))
plt.colorbar()

In [ ]:
out = model.predict(xs_in[0])
out.shape

In [ ]:
plt.imshow(np.squeeze(out[0]))
plt.colorbar()

In [ ]:
out = model.predict(xs_in[0])
out.shape

In [ ]:
plt.imshow(np.squeeze(out[0]))
plt.colorbar()

In [ ]:
bins = np.linspace(1, 10000, 100)

y = np.squeeze(ys[0]).flatten()
y_pred = np.squeeze(out[0]).flatten()
plt.hist(y, bins, alpha=0.5, label='y', color='r')
plt.hist(y_pred, bins, alpha=0.5, label='y_pred', color='b')
plt.legend(loc='upper right')
plt.show()

In [ ]:
np.unique(out[0])

In [14]:
run.finish()

epoch,29
d_loss,0.34739
g_loss,0.70817
mse_loss,0.0
_runtime,756
_timestamp,1621351076
_step,65


epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
d_loss,▂▂▄▆▆▆▇▇▆▆▆▆▆▇▇█▇█▇██▇▆▇▆▅▄▃▂▃▂▁▁
g_loss,▆▅▃▂▁▁▂▂▂▂▃▃▂▂▁▁▁▁▂▁▂▂▃▃▄▅▆▇███▇▇
mse_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████
_timestamp,▁▁▁▁▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


In [ ]:
model.save("gan_model")

In [ ]:
model.generator.save("generator")

In [ ]:
# Test generator
start_dt = datetime(2019,6,6,0,0)
end_dt =  datetime(2019,6,6,0,5)

list_IDs = get_list_IDs(start_dt, end_dt, x_seq_size, y_seq_size, filter_no_rain=filter_no_rain)
generator = DataGenerator(list_IDs, batch_size=1, x_seq_size=x_seq_size, 
                                       y_seq_size=y_seq_size, normalize_y=False)
for i in generator:
    x,y = i
    print(x.shape, y.shape)

In [ ]:
y_pred = model.generator(x)
y_pred.shape

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(np.squeeze(y))
plt.colorbar()
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(np.squeeze(y_pred))
plt.colorbar()
plt.show()

In [ ]:
plt.hist(y_pred.numpy().flatten())

In [ ]:
model.discriminator(y_pred)